In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import pickle
from nltk.stem import WordNetLemmatizer
import re

In [2]:
# Load the dataset
tweets = pd.read_pickle("cleaned_tweets_v1.pkl")
X = tweets['cleaned_tweets']
y = tweets['label']

In [3]:
tweets.head()

,label,tweet,cleaned_tweets,cleaned_tweets_without_stopwords
0,1,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beauti...,fingerprint pregnancy test android apps beauti...
1,1,Finally a transparant silicon case ^^ Thanks t...,finally transparant silicon case thanks uncle ...,finally transparant silicon case thanks uncle ...
2,1,We love this! Would you go? #talk #makememorie...,love this would you talk makememories unplug r...,love talk makememories unplug relax iphone sma...
3,1,I'm wired I know I'm George I was made that wa...,wired know george wa made that way iphone cute...,wired know george way iphone cute daventry home
4,0,What amazing service! Apple won't even talk to...,what amazing service apple will not even talk ...,amazing service apple talk question unless pay...


In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train.shape

(6336,)

In [6]:
y_train.value_counts(normalize=True)

label
1    0.74416
0    0.25584
Name: proportion, dtype: float64

In [7]:
y_test.value_counts(normalize=True)

label
1    0.744318
0    0.255682
Name: proportion, dtype: float64

In [8]:
# Create vocabulary from the trianing set only
counter = Counter()
for tweet in X_train:
    counter.update(tweet.split())

vocab = set(counter.keys())

In [9]:
# vocab

In [10]:
# Save vocabulary
with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

with open('vocab.txt', 'w') as f:
    for word in vocab:
        f.write(f"{word}\n")

print(f"Vocabulary size: {len(vocab)}")
print("Top 10 tokens:", counter.most_common(10))

Vocabulary size: 13491
Top 10 tokens: [('iphone', 3354), ('apple', 2326), ('the', 1601), ('samsung', 1147), ('and', 1016), ('new', 947), ('you', 912), ('twitter', 898), ('for', 855), ('phone', 851)]


In [11]:
# Find the maximum length of tweets
max_length = max(len(tweet.split()) for tweet in X_train)

In [12]:
max_length

45

In [13]:
print(list(vocab)[:10])

['bamboo', 'nancy', 'apocalyptic', 'bolly', 'feed', 'wtofg', 'makan', 'fucktheiphone', 'tqyuca', 'neverland']


In [14]:
# Create word to index mapping
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}  # dict comprehension
word_to_idx['<PAD>'] = 0

In [15]:
# word_to_idx

In [16]:
# Convert tweets to sequences of indices
def text_to_sequence(text):
    return [word_to_idx.get(word, 0) for word in text.split()]

X_train_seq = [text_to_sequence(tweet) for tweet in X_train]
X_test_seq = [text_to_sequence(tweet) for tweet in X_test]

In [17]:
len(X_train.iloc[0].split()), X_train.iloc[0]

(16,
 'fucking done with the apple iphone never fucking again lost everything fuck you apple die apple')

In [18]:
len(X_train_seq[0])

16

In [19]:
print(X_train_seq[0])

[12894, 5188, 2413, 3583, 3190, 9277, 5914, 12894, 7487, 2372, 46, 7887, 3401, 3190, 3517, 3190]


In [20]:
word_to_idx["wash"]

3041

In [21]:
word_to_idx["wash"]

3041

In [22]:
len(X_train_seq[0]), len(X_train_seq[1]), len(X_train_seq[2])

(16, 6, 10)

In [23]:
# Pad sequences
def pad_sequences(sequences, maxlen):
    return [seq + [0]*(maxlen - len(seq)) if len(seq) < maxlen else seq[:maxlen] for seq in sequences]

X_train_pad = pad_sequences(X_train_seq, max_length)
X_test_pad = pad_sequences(X_test_seq, max_length)

In [24]:
print(X_train_pad[0])

[12894, 5188, 2413, 3583, 3190, 9277, 5914, 12894, 7487, 2372, 46, 7887, 3401, 3190, 3517, 3190, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
print(X_train_pad[1])

[3070, 2505, 9337, 5769, 7768, 9277, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
# Convert to PyTorch tensors
X_train_tensor = torch.LongTensor(X_train_pad)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.LongTensor(X_test_pad)
y_test_tensor = torch.FloatTensor(y_test.values)

# Create dataset and dataloader
class TweetDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
train_dataset = TweetDataset(X_train_tensor, y_train_tensor)
test_dataset = TweetDataset(X_test_tensor, y_test_tensor)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [28]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, kernel_size, max_length):
        super(CNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(num_filters * ((max_length - kernel_size + 1)//2), 10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10, 1)
        
    def forward(self, x):
        embedded = self.embedding(x).permute(0, 2, 1)
        conv_out = torch.relu(self.conv1(embedded))
        pooled = self.pool(conv_out)
        flattened = pooled.view(pooled.size(0), -1)
        fc1_out = torch.relu(self.fc1(flattened))
        dropped = self.dropout(fc1_out)
        fc2_out = self.fc2(dropped)
        return torch.sigmoid(fc2_out).squeeze()

In [29]:
"<PAD>" in list(vocab)

False

In [30]:
"lte" in list(vocab)

True

In [31]:
list(vocab)[0], list(vocab)[-1]

('bamboo', 'survive')

In [42]:
# Hyperparameters
embedding_dim = 50
num_filters = 32
kernel_size = 4
vocab_size = len(vocab) + 1  # +1 for padding token

In [43]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model = CNNModel(vocab_size, embedding_dim, num_filters, kernel_size, max_length).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    cnn_model.train()
    epoch_loss = 0
    epoch_acc = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        epoch_acc += (preds == labels).sum().item()
    
    epoch_loss /= len(train_dataset)
    epoch_acc /= len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

# Evaluation
cnn_model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        test_acc += (preds == labels).sum().item()

test_loss /= len(test_dataset)
test_acc /= len(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.2f}%")

# Save the model
torch.save(cnn_model.state_dict(), 'cnn_model.pth')

Epoch [1/10], Loss: 0.5711, Accuracy: 74.38%
Epoch [2/10], Loss: 0.4836, Accuracy: 74.89%
Epoch [3/10], Loss: 0.3984, Accuracy: 79.14%
Epoch [4/10], Loss: 0.3482, Accuracy: 81.09%
Epoch [5/10], Loss: 0.3197, Accuracy: 81.44%
Epoch [6/10], Loss: 0.2819, Accuracy: 86.38%
Epoch [7/10], Loss: 0.2593, Accuracy: 87.55%
Epoch [8/10], Loss: 0.2268, Accuracy: 88.90%
Epoch [9/10], Loss: 0.2065, Accuracy: 89.87%
Epoch [10/10], Loss: 0.1874, Accuracy: 90.15%
Test Loss: 0.3654, Test Accuracy: 85.35%


In [40]:
# Tweet cleaning function
lemmatizer = WordNetLemmatizer()

def tweet_cleaner(raw_tweet):
    cleaned_tweet = re.sub("@[A-Za-z0-9]+", "", raw_tweet)
    cleaned_tweet = re.sub("#", "", cleaned_tweet)
    cleaned_tweet = re.sub(r"http\S+", "", cleaned_tweet)
    cleaned_tweet = re.sub(r"[^a-zA-Z]", " ", cleaned_tweet)
    cleaned_tweet = cleaned_tweet.lower().strip()
    cleaned_tweet = [token for token in cleaned_tweet.split() if len(token) > 2]
    new_sent = ' '.join([lemmatizer.lemmatize(token) for token in cleaned_tweet])
    return new_sent.strip()

# Prediction function
def predict_tweet(model, tweet):
    model.eval()
    cleaned_tweet = tweet_cleaner(tweet)
    seq = text_to_sequence(cleaned_tweet)
    padded_seq = pad_sequences([seq], max_length)
    input_tensor = torch.LongTensor(padded_seq).to(device)
    with torch.no_grad():
        output = model(input_tensor)
    prediction = (output >= 0.5).float()
    return output.item(), prediction.item()

In [44]:
# Example prediction
new_tweet = "this phone is disgusting! #iphone @Apple"
probability, prediction = predict_tweet(cnn_model, new_tweet)
print(f"Probability: {probability:.4f}, Prediction: {prediction}")

Probability: 0.3445, Prediction: 0.0


In [45]:
import torch
import torch.nn as nn

class ComplexCNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, kernel_sizes, max_length):
        super(ComplexCNNModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # First set of convolutional layers
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_sizes[0])
        self.bn1 = nn.BatchNorm1d(num_filters)
        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_sizes[1])
        self.bn2 = nn.BatchNorm1d(num_filters)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        
        # Second set of convolutional layers
        self.conv3 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters*2, kernel_size=kernel_sizes[2])
        self.bn3 = nn.BatchNorm1d(num_filters*2)
        self.conv4 = nn.Conv1d(in_channels=num_filters*2, out_channels=num_filters*2, kernel_size=kernel_sizes[3])
        self.bn4 = nn.BatchNorm1d(num_filters*2)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        
        # Calculate the size of the output from the last pooling layer
        self.feature_size = self._get_conv_output_size(max_length)
        
        # Fully connected layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 1)  # Output layer
        
    def _get_conv_output_size(self, length):
        # Helper function to calculate the output size of the last pooling layer
        with torch.no_grad():
            dummy_input = torch.zeros(1, self.embedding.embedding_dim, length)
            dummy_input = self.conv1(dummy_input)
            dummy_input = self.conv2(dummy_input)
            dummy_input = self.pool1(dummy_input)
            dummy_input = self.conv3(dummy_input)
            dummy_input = self.conv4(dummy_input)
            dummy_input = self.pool2(dummy_input)
            return dummy_input.numel()
        
    def forward(self, x):
        # Embedding layer
        x = self.embedding(x).permute(0, 2, 1)  # (batch, embed_dim, seq_len)
        
        # First set of conv layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool1(x)
        
        # Second set of conv layers
        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = self.pool2(x)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return torch.sigmoid(x).squeeze()

In [46]:
# Hyperparameters
vocab_size = len(vocab) + 1  # +1 for padding token
embedding_dim = 50
num_filters = 64
kernel_sizes = [3, 3, 3, 3]  # for the four conv layers
max_length = 45  # or whatever your max sequence length is

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
complex_cnn_model = ComplexCNNModel(vocab_size, embedding_dim, num_filters, kernel_sizes, max_length).to(device)

# Print model summary
print(complex_cnn_model)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(complex_cnn_model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    complex_cnn_model.train()
    epoch_loss = 0
    epoch_acc = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = complex_cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        epoch_acc += (preds == labels).sum().item()
    
    epoch_loss /= len(train_dataset)
    epoch_acc /= len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

# Evaluation
complex_cnn_model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = complex_cnn_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        test_acc += (preds == labels).sum().item()

test_loss /= len(test_dataset)
test_acc /= len(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.2f}%")

# Save the model
torch.save(complex_cnn_model.state_dict(), 'complex_cnn_model.pth')

ComplexCNNModel(
  (embedding): Embedding(13492, 50, padding_idx=0)
  (conv1): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (

In [48]:
from torch.utils.data import Dataset, DataLoader, random_split

# Hyperparameters
vocab_size = len(vocab) + 1  # +1 for padding token
embedding_dim = 50
num_filters = 64
kernel_sizes = [3, 3, 3, 3]
max_length = 45
batch_size = 64
num_epochs = 20
learning_rate = 0.001
weight_decay = 1e-3  # L2 regularization

# Split the training data into training and validation sets
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ComplexCNNModel(vocab_size, embedding_dim, num_filters, kernel_sizes, max_length).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Early stopping
best_val_loss = float('inf')
patience = 3
epochs_no_improve = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_acc = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        train_acc += (preds == labels).sum().item()
    
    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset)
    
    # Validation
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = (outputs >= 0.5).float()
            val_acc += (preds == labels).sum().item()
    
    val_loss /= len(val_dataset)
    val_acc /= len(val_dataset)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping!")
            break

# Load best model and evaluate on test set
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        test_acc += (preds == labels).sum().item()

test_loss /= len(test_dataset)
test_acc /= len(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.2f}%")

Epoch [1/20], Train Loss: 0.4897, Train Acc: 76.09%, Val Loss: 0.4320, Val Acc: 76.36%
Epoch [2/20], Train Loss: 0.3632, Train Acc: 83.14%, Val Loss: 0.3973, Val Acc: 80.56%
Epoch [3/20], Train Loss: 0.2899, Train Acc: 87.64%, Val Loss: 0.7976, Val Acc: 80.74%
Epoch [4/20], Train Loss: 0.2262, Train Acc: 90.66%, Val Loss: 0.3874, Val Acc: 82.31%
Epoch [5/20], Train Loss: 0.1750, Train Acc: 93.43%, Val Loss: 0.5031, Val Acc: 82.14%
Epoch [6/20], Train Loss: 0.1152, Train Acc: 95.87%, Val Loss: 0.7345, Val Acc: 81.79%
Epoch [7/20], Train Loss: 0.0911, Train Acc: 96.86%, Val Loss: 0.7322, Val Acc: 81.09%
Early stopping!
Test Loss: 0.4136, Test Accuracy: 81.94%


In [57]:
# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        return torch.sigmoid(self.fc(hidden)).squeeze()

# Initialize LSTM model
hidden_dim = 100
lstm_model = LSTMModel(vocab_size, embedding_dim, hidden_dim, 1).to(device)

# Training loop for LSTM model
optimizer = optim.Adam(lstm_model.parameters(), lr=0.0005)
num_epochs = 10

for epoch in range(num_epochs):
    lstm_model.train()
    epoch_loss = 0
    epoch_acc = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = lstm_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        epoch_acc += (preds == labels).sum().item()
    
    epoch_loss /= len(train_dataset)
    epoch_acc /= len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

# Evaluation of LSTM model
lstm_model.eval()
test_loss = 0
test_acc = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = lstm_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        test_acc += (preds == labels).sum().item()

test_loss /= len(test_dataset)
test_acc /= len(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.2f}%")

# Save the LSTM model
torch.save(lstm_model.state_dict(), 'lstm_model.pth')

Epoch [1/10], Loss: 0.5947, Accuracy: 70.94%
Epoch [2/10], Loss: 0.5231, Accuracy: 74.25%
Epoch [3/10], Loss: 0.4571, Accuracy: 73.46%
Epoch [4/10], Loss: 0.4333, Accuracy: 77.61%
Epoch [5/10], Loss: 0.4288, Accuracy: 74.70%
Epoch [6/10], Loss: 0.4013, Accuracy: 80.02%
Epoch [7/10], Loss: 0.3629, Accuracy: 83.65%
Epoch [8/10], Loss: 0.3421, Accuracy: 84.95%
Epoch [9/10], Loss: 0.3104, Accuracy: 86.81%
Epoch [10/10], Loss: 0.2851, Accuracy: 88.25%
Test Loss: 0.3737, Test Accuracy: 83.90%
